# Constraint Programming and Automatic Differentiation for Engineering Design
***
### Luke McCulloch, PhD

\newcommand{\volDisp}{\mathrm{Subme}}

## Automation
  - Here I pick a problem to automate and tell you a little about it
  
## Automated Design
  - Functional minimization (extrema finding) sets the stage for what's to come

## Automatic Differentiation 
  - Programming without (as much) programming
  - Gradient decent without gradient coding
  - Gradients and Hessians with 
    * no finite differences
    * no symbolic derivatives
    * values exact to floating point
  
## Interval constraint programming  
  - Design without (as much) _search_
  - Because we eliminate infeasible designs
  
## Results
  - If your going to solve a problem, eventually you have to... solve a problem.
  - I picked an easy one so I could finish with reasonable effort.

A picked problem:  automate the generation of this kind of thing:

![title](fig/RapidHull.png)

(NURBS early stage hullform (OSV type) in demo in Rhino, human created)

In [26]:
import warnings
warnings.filterwarnings('ignore') #sorry, I've got old code that needs updating -- shame on me!



This is called,

## Early Stage Ship Hull Design

Why would we want to work on this?

* One of a kind designs
* Less standardization than other industries
* No prototypes, less chance to iron out issues

![title](fig/DesignChallenges.png)

# Use Computation to Reduce Uncertainty

![title](fig/TradDesignVsOptimization.png)

# Automate Design Exploration

## In Ship Design Automation, 
## geometry generation has been a major bottleneck
* Physics solvers are "mature" (enough I guess)
* we are _getting_ _better_ at meshing (not great yet)
* we generate designs by the engineer-week (month? $$$)

## Form Parameter Design

* State of the art for early stage ship design
* Works by constrained optimization, minimizing some energy fuctional with constraints.

### It turns design into a functional to be minimized.

## What kinds of properties do we care about?
$$ 
\begin{align*}
\nabla &: \text{Displacement} \\
L_{\text{WL}} &: \text{waterline length} \\
B &: \text{beam} \\
D &: \text{depth} \\
etc  ...
\end{align*}
$$
![title](fig/Rules/Cb.png)
### Basic sizing, dimensions, and ...

## What kinds of properties do we care about?

## Properties we care about:  differential / transitional
![title](fig/HullProperties/DifferentialProperties.png)

## Properties we care about: sectional areas
![title](fig/midship_inplace.png)

## Properties we care about:  sectional area
![title](fig/HullProperties/MidshipCoeff.png)

## What kinds of properties do we care about?

## Properties we care about:  volumetric / represented as area again!
![title](fig/HullProperties/SAC.png)

## What kinds of properties do we care about?

## Properties we care about: curvature
![title](fig/HullProperties/ManualDeformDerivativeIssue.png)

## Control Properties with: Form Parameter Design
![title](fig/FPD/FPDsummary.png)

## How do we Make a B-spline curve which conforms to constraints?
![title](fig/FPD/cagd15curvedefBig.png)

Find a two dimensional open B-spline curve

\begin{equation}
\underline{q}(t)\;=\;\sum_{i=0}^{n}\underline{V}_{i}N_{i,k}(t) 
\label{eq:optproblem}
\end{equation}

such that a flexible set of selected form parameters 
is met and, in addition, the curve is considered "good" 
in terms of a problem-oriented measure.

Find a B-spline curve of given order $k$, number of vertices $n+1$, and specified knot 
vector $\underline{T}$ 

Problem:
	
$$
F = f + \sum_{j \in \left[ \forall h \right]}  \lambda_j h_j    
$$
	
	
$$F = F\left(\underline{V},\underline{\lambda}\right)$$
	
Solution:
	
$$\underline{\nabla} F = [0,0,0,0.... ]^T$$
	
Find control vertices and Lagrange multipliers such that 
the gradient of $F$ is $\underline{0}$
	
Solve via Newton's method.

In [27]:
import numpy as np #linear algebra
np.set_printoptions(precision=3)

In [28]:
import relational_lsplines as rsp #my stuff

In [29]:
curve = rsp.curve #Bspline curve and surface module


automatic_differentiation = rsp.automatic_differentiation #automatic_differentiation "module"
                    # simple fwd mode with vectors and intervals supported

adObjectMaker = automatic_differentiation.adObjectMaker

ad = automatic_differentiation.ad                   #automatic differentiation class


## How to Auto Diff

In principle, for automatic differentiation on a design space, you need only very simple things

In [30]:
num_in_vec = 2  # problem space size, in this toy example

identity = np.matrix(np.identity(num_in_vec)) # [2x2] identity matrix

nullmatrix = np.matrix(np.zeros((num_in_vec,num_in_vec),float)) # [2x2] matrix of 0s

In [31]:
x0 = ad(0.5, 
        grad  = identity[0], 
        hess  = nullmatrix, 
        of_scalars=True)

print 'x0.value = ',x0.value
print 'x0.grad  = ', x0.grad
print 'x0.hess  = '
print x0.hess

x0.value =  0.5
x0.grad  =  [[1. 0.]]
x0.hess  = 
[[0. 0.]
 [0. 0.]]


### This use of the identity matrix to appoint gradients to free variables of the design space is reminecint of the adjacency matrix from meshing.

* and we all know that the connectivity of a mesh, as defined by it's adjacency matrices on vertices, edges, and faces, leads directly to the exterior derivative of said quantities.
* coincidence?  Heck no!



So we have created a variable, x0

We gave it a gradient and hessian associated with it, 
with sizes denoted by the problem space. 

Later, this part will be automated too.


In [32]:
x1 = ad(2.0, 
        grad  = identity[1], 
        hess  = nullmatrix, 
        of_scalars=True)

print ''
print 'x1.value = ',x1.value
print 'x1.grad  = ', x1.grad
print 'x1.hess  = '
print x1.hess





x1.value =  2.0
x1.grad  =  [[0. 1.]]
x1.hess  = 
[[0. 0.]
 [0. 0.]]


In [33]:
test = x0 + x1**2

print '\ntest value'
print test.value

print '\ntest gradient'
print test.grad

print '\ntest hessian'
print test.hess


test value
4.5

test gradient
[[1. 4.]]

test hessian
[[0. 0.]
 [0. 2.]]


## (Interval) Constraint Programming

> A designer knows he has achieved perfection not when there is nothing left to add, but when there is nothing left to take away. 

\-Antoine de Saint-Exupéry

Sorry, that's melodramatic.  
I just fed my data into a slide generator algorithm.  
I didn't have time to clean it up after.

With a section curve, one constraint is that the area of the section should fit in a box:

<img src="fig/midship_section.png" alt="Drawing" style="width: 400px;"/>